# Implement Linear Regression with Forward & Backward Features Selection (MSE metric)
__Author__ : Mohammad Rouintan , 400222042

__Course__ : Undergraduate Machine Learning Course

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from sklearn.base import clone
from sklearn.metrics import mean_squared_error

## Problem
Implement Forward and Backward Feature selection algorithms from scratch with MSE as the metric.


### Residual Sum Of Squares (RSS)
$$
\begin{align}
RSS &= e_{1}^{2} + e_{2}^{2} + \dots + e_{n}^{2} \\
RSS &= (y_{1} - \hat{y}_{1})^{2} + (y_{2} - \hat{y}_{2})^{2} + \dots + (y_{n} - \hat{y}_{n})^{2}
\end{align}
$$


### Forward Selection
Forward Selection chooses a subset of the predictor variables for the final model.

We can do forward stepwise in context of linear regression whether n is less than p or n is greater than p.

Forward selection is a very attractive approach, because it's both tractable and it gives a good sequence of models.

1.  Start with a null model. The null model has no predictors, just one intercept (The mean over Y).
2.  Fit p simple linear regression models, each with one of the variables in and the intercept. So basically, you just search through all the single-variable models the best one (the one that results in the lowest residual sum of squares). You pick and fix this one in the model.
3.  Now search through the remaining p minus 1 variables and find out which variable should be added to the current model to best improve the residual sum of squares.
4.  Continue until some stopping rule is satisfied, for example when all remaining variables have a p-value above some threshold.

### How to determine the most significant variable at each step ?
The most significant variable can be chosen so that, when added to the model:
1. It has the smallest p-value, or
2. It provides the highest increase in R2, or
3. It provides the highest drop in model RSS (Residuals Sum of Squares) compared to other predictors under consideration.

### How to choose a stopping rule ?
The stopping rule is satisfied when all remaining variables to consider have a p-value larger than some specified threshold, if added to the model.

When we reach this state, forward selection will terminate and return a model that only contains variables with p-values < threshold.

### How to determine the threshold ?
The threshold can be:

1. A fixed value (for instance: 0.05 or 0.2 or 0.5)
2. Determined by AIC (Akaike Information Criterion)
3. Determined by BIC (Bayesian Information Criterion)

If we choose a fixed value, the threshold will be the same for all variables.
However, if we let AIC or BIC automatically determine the threshold, it will be different for each variable.

### How does AIC determine the threshold ?
AIC chooses the threshold according to how many degrees of freedom the variable under consideration has.

Take for example the case of a binary variable (by definition it has 1 degree of freedom): According to AIC, if this variable is to be included in the model, it needs to have a p-value < 0.157.

The more degrees of freedom a variable has, the lower the threshold will be.

### How does BIC determine the threshold ?
BIC chooses the threshold according to the effective sample size n.

For instance, for n = 20, a variable will need a p-value < 0.083 in order to enter the model.

The larger n is, the lower the threshold will be.

BIC is a more restrictive criterion than AIC and so yields smaller models. Therefore it is only recommended when working with large sample sizes — where the sample size (or number of events in case of logistic regression) exceeds 100 per independent variable [Heinze et al.].

Note that both AIC (and BIC) can be applied to the pooled degrees of freedom of all unselected predictors. But applying it to individual variables (like we described above) is far more prevalent in practice.

## Forward Selection

In [61]:
class ForwardSelection():
    def __init__(self, algorithm, k_features):
        self.algorithm = clone(algorithm)
        self.k_features = k_features
    
    def calculate_mse(self, x_train, x_test, y_train, y_test, index):
        self.algorithm.fit(x_train[:, index], y_train)
        predicted_y = self.algorithm.predict(x_test[:, index])
        mse = mean_squared_error(y_test, predicted_y)
        return mse

    def fit(self, x_train, x_test, y_train, y_test):
        features = tuple(np.arange(x_train.shape[1]))
        number_of_features = len(features)
        self.subsets = list()
        self.scores = list()
        self.indices = list()

        subset = list()
        score = list()
        for feature in combinations(features, r=1):
            score.append(self.calculate_mse(x_train, x_test, y_train, y_test, feature))
            subset.append(feature)

        best_score = np.argmin(score)
        self.scores.append(score[best_score])
        self.indices = list(subset[best_score])
        self.subsets.append(self.indices)

        counter = 1
        for i in range(1, self.k_features):
            subset = list()
            score = list()

            for index in range(number_of_features):
                if index not in self.indices:
                    temp = list(self.indices)
                    temp.append(index)
                    score.append(self.calculate_mse(x_train, x_test, y_train, y_test, temp))
                    subset.append(temp)
            
            best_score = np.argmin(score)
            self.scores.append(score[best_score])
            self.indices = list(subset[best_score])
            self.subsets.append(self.indices)


    
    def transform(self, x):
        return x[:, self.indices]

In [62]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
X, y = make_regression(n_samples=500, n_features=10, n_informative=8, noise=15, random_state=40)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [63]:
from sklearn.linear_model import LinearRegression
Linear_Regression = LinearRegression()

In [65]:
forward_selection = ForwardSelection(Linear_Regression, k_features=6)
forward_selection.fit(x_train, x_test, y_train, y_test)
x_train_fixed = forward_selection.transform(x_train)
x_test_fixed = forward_selection.transform(x_test)

In [67]:
Linear_Regression.fit(x_train_fixed, y_train)
predicted_y = Linear_Regression.predict(x_test_fixed)

mse = mean_squared_error(predicted_y, y_test)
accuracy = Linear_Regression.score(x_test_fixed, y_test)
print(f"The mean squared error of the optimal model is {mse:.5f}")
print(f"The accuracy score of the optimal model is {accuracy:.5f}")

The mean squared error of the optimal model is 2038.13611
The accuracy score of the optimal model is 0.95107


### Backward Elimination
Unlike forward stepwise selection, it begins with the full least squares model containing all p predictors, and then iteratively removes the least useful predictor, one-at-a-time.

In order to be able to perform backward elimination, we need to be in a situation where we have more observations than variables because we can do least squares regression when n is greater than p. If p is greater than n, we cannot fit a least squares model. It's not even defined.

1. Start with all variables in the model.
2. Remove the variable with the largest p-value | that is, the variable that is the least statistically significant.
3. The new (p - 1)-variable model is t, and the variable with the largest p-value is removed.
4. Continue until a stopping rule is reached. For instance, we may stop when all remaining variables have a significant p-value defined by some significance threshold.

### How to determine the least significant variable at each step ?
The least significant variable is a variable that:

1. Has the highest p-value in the model, or
2. Its elimination from the model causes the lowest drop in R2, or
3. Its elimination from the model causes the lowest increase in RSS (Residuals Sum of Squares) compared to other predictors

### How to choose a stopping rule ?
The stopping rule is satisfied when all remaining variables in the model have a p-value smaller than some pre-specified threshold.

When we reach this state, backward elimination will terminate and return the current step’s model.

### How to determine the threshold ?
As with forward selection, the threshold can be:

1. A fixed value (for instance: 0.05 or 0.2 or 0.5)
2. Determined by AIC (Akaike Information Criterion)
3. Determined by BIC (Bayesian information criterion)

In [ ]:
class BackwradElimination():
    def __init__(self, algorithm, k_features):
        self.algorithm = clone(algorithm)
        self.k_features = k_features
    
    def calculate_mse(self, x_train, x_test, y_train, y_test, index):
        self.algorithm.fit(x_train[:, index], y_train)
        predicted_y = self.algorithm.predict(x_test[:, index])
        mse = mean_squared_error(y_test, predicted_y)
        return mse

    def fit(self, x_train, x_test, y_train, y_test):
        features = tuple(np.arange(x_train.shape[1]))
        number_of_features = len(features)
        self.subsets = list()
        self.scores = list()
        self.indices = list()

        subset = list()
        score = list()
        for feature in combinations(features, r=1):
            score.append(self.calculate_mse(x_train, x_test, y_train, y_test, feature))
            subset.append(feature)

        best_score = np.argmin(score)
        self.scores.append(score[best_score])
        self.indices = list(subset[best_score])
        self.subsets.append(self.indices)

        counter = 1
        for i in range(1, self.k_features):
            subset = list()
            score = list()

            for index in range(number_of_features):
                if index not in self.indices:
                    temp = list(self.indices)
                    temp.append(index)
                    score.append(self.calculate_mse(x_train, x_test, y_train, y_test, temp))
                    subset.append(temp)
            
            best_score = np.argmin(score)
            self.scores.append(score[best_score])
            self.indices = list(subset[best_score])
            self.subsets.append(self.indices)


    
    def transform(self, x):
        return x[:, self.indices]

## Conclusion for this problem
Write a conclusion and references which you've used in your homework